In [1]:
args = {
    'path': '/data0/pretrained-models/deepseek-coder-7b-base-v1.5',
    'device': 'cuda:1',
    'mm_projector_type': 'linear',
    'encoder_hidden_size': 4096,
    'decoder_hidden_size': 4096
}

In [2]:
import torch
from torch import nn
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained(args['path']).to(device)
tokenizer = AutoTokenizer.from_pretrained(args['path'])

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(102400, 4096)
    (layers): ModuleList(
      (0-29): 30 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head)

In [3]:
from decoder import TableDecoder
from projector import Projector



config: {'path': '/data0/pretrained-models/deepseek-coder-7b-base-v1.5', 'device': 'cuda:1', 'mm_projector_type': 'linear', 'encoder_hidden_size': 4096, 'decoder_hidden_size': 4096}
proj_type linear


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [16]:
decoder = decoder.to(device)

In [17]:
inp = torch.tensor(tokenizer.encode('114514'), device = device).unsqueeze(0)
tab = torch.tensor(tokenizer.encode('1919810'), device = device).unsqueeze(0)
decoder(input_ids = inp, table_embeds = model.get_input_embeddings()(tab))

CausalLMOutputWithPast(loss=None, logits=tensor([[[ -3.3266,  -3.1734,  -2.5148,  ...,  -5.8050,  -5.8058,  -5.7851],
         [ -5.3963,  -5.1817,  -7.7607,  ..., -16.2539, -16.2620, -16.2623],
         [ -5.4716,  -5.0509,  -8.6906,  ..., -17.9581, -17.8856, -17.9384],
         ...,
         [ -9.9950,  -8.2155,  -8.0499,  ..., -18.6914, -18.6550, -18.6339],
         [ -9.8995,  -8.2798, -10.0717,  ..., -19.0142, -18.9756, -18.9679],
         [ -6.4253,  -6.2526,  -8.0231,  ..., -19.8761, -19.8941, -19.8040]]],
       device='cuda:1', grad_fn=<UnsafeViewBackward0>), past_key_values=((tensor([[[[ 2.4846e-01, -3.8195e-01, -8.6933e-02,  ...,  1.7131e-01,
           -5.0102e-01, -3.6520e-01],
          [ 1.6370e-01,  3.3236e-02, -8.8762e-02,  ..., -3.6705e-02,
           -1.6247e-01, -4.6747e-01],
          [ 2.0209e-01, -2.2612e-01, -6.4668e-02,  ..., -1.4900e-01,
            5.1338e-01, -5.1562e-01],
          ...,
          [-5.9488e-01, -2.0351e-01, -3.3428e-01,  ..., -4.3795e-01,
  

In [2]:
args = {
    "base_model_name_or_path": "/mnt/data/models/deepseek-ai/deepseek-coder-1.3b-instruct",
    "device": "cuda",
    "mm_projector_type": "mlp2x_gelu",
    "encoder_hidden_size": 384,
    "decoder_hidden_size": 2048,
    "n_tokens": 10,
    "torch_dtype": "float32"
}
# decoder = TableDecoder(model, args)

In [7]:
from model.decoder import TableDecoderConfig, TableDecoder
config = TableDecoderConfig(**args)
decoder = TableDecoder.from_pretrained(config=config)
next(decoder.projector.parameters())

Parameter containing:
tensor([[-0.0017,  0.0201, -0.0159,  ...,  0.0362, -0.0015,  0.0428],
        [-0.0071, -0.0420,  0.0162,  ..., -0.0435,  0.0316, -0.0274],
        [ 0.0332,  0.0236, -0.0364,  ...,  0.0208,  0.0182,  0.0080],
        ...,
        [ 0.0041,  0.0187,  0.0431,  ..., -0.0112, -0.0454, -0.0407],
        [ 0.0275,  0.0081,  0.0332,  ..., -0.0008, -0.0013, -0.0197],
        [ 0.0288, -0.0020,  0.0062,  ..., -0.0479,  0.0236,  0.0287]],
       requires_grad=True)

TableDecoder(
  (projector): Projector(
    (model): Sequential(
      (0): Linear(in_features=384, out_features=2048, bias=True)
      (1): GELU(approximate='none')
      (2): Linear(in_features=2048, out_features=2048, bias=True)
    )
  )
  (model): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(32256, 2048)
      (layers): ModuleList(
        (0-23): 24 x LlamaDecoderLayer(
          (self_attn): LlamaSdpaAttention(
            (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
            (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
            (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
            (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
            (rotary_emb): LlamaLinearScalingRotaryEmbedding()
          )
          (mlp): LlamaMLP(
            (gate_proj): Linear(in_features=2048, out_features=5504, bias=False)
            (up_proj): Linear(in_features=2048, out_fe

In [1]:
import os
def find_correct_case_file_name(path, name):
    try:
        ls = os.listdir(path)
        for gt in ls:
            gt = gt.strip('.csv')
            if gt.lower() == name.lower():
                return gt
        # 找为子串的
        for gt in ls:
            gt = gt.strip('.csv')
            if name.lower() in gt.lower():
                return gt
        return None
    except:
        return None
import json
with open('/mnt/data/gxj/code/dataset/train_new.json', "r") as f:
    data = json.load(f)

In [2]:
db_path = '/home/xjgu/spider/database_csv'
for conv in data:
    db_id, table = conv['db_id'], conv['table']
    res = find_correct_case_file_name(os.path.join(db_path, db_id), table)
    if res == None:
        print(db_id, table)